In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


In [9]:
# Load the data
df = pd.read_excel('consolidated_workbook.xlsx')

print(df.columns)

Index(['        Year \n ', 'Domestic liabilities of the Centre',
       'External liabilities of the Centre',
       'Total liabilities of the Centre ', 'Gross fiscal deficit',
       'Revenue deficit', 'Tax Revenue (Net)', 'Direct Tax (Net)',
       'Indirect Tax', 'Non-Tax Revenue', 'Revenue Receipts',
       'Capital Receipts', 'Total Receipts',
       'Volume of Central Govt Dated Securities', 'Rate of inflation',
       'Change from previous year', 'CRR', 'Repo Rate'],
      dtype='object')


In [10]:


# Define the feature and target columns
predictors = ['Domestic liabilities of the Centre', 'External liabilities of the Centre', 'Total liabilities of the Centre ',
              'Gross fiscal deficit', 'Revenue deficit', 'Tax Revenue (Net)', 'Direct Tax (Net)', 'Indirect Tax',
              'Non-Tax Revenue', 'Revenue Receipts', 'Capital Receipts', 'Total Receipts', 'Volume of Central Govt Dated Securities']

control_variables = ['CRR', 'Repo Rate']
target_inflation = 'Rate of inflation'
target_change_inflation = 'Change from previous year'

# Combine predictors and control variables
X = df[predictors + control_variables]

# Targets
y_inflation = df[target_inflation]
y_change_inflation = df[target_change_inflation]

# Standardize the predictors and control variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [11]:
# Split the data into training and testing sets for both target variables
X_train_infl, X_test_infl, y_train_infl, y_test_infl = train_test_split(X_scaled, y_inflation, test_size=0.2, random_state=42)
X_train_change_infl, X_test_change_infl, y_train_change_infl, y_test_change_infl = train_test_split(X_scaled, y_change_inflation, test_size=0.2, random_state=42)


In [19]:
# Adding a constant to the model (intercept)
X_train_infl_sm = sm.add_constant(X_train_infl)
X_test_infl_sm = sm.add_constant(X_test_infl)

X_train_change_infl_sm = sm.add_constant(X_train_change_infl)
X_test_change_infl_sm = sm.add_constant(X_test_change_infl)

# OLS Regression for rate of inflation
ols_infl = sm.OLS(y_train_infl, X_train_infl_sm).fit()
y_pred_infl_sm = ols_infl.predict(X_test_infl_sm)
mse_infl_sm = mean_squared_error(y_test_infl, y_pred_infl_sm)

# OLS Regression for change in inflation
ols_change_infl = sm.OLS(y_train_change_infl, X_train_change_infl_sm).fit()
y_pred_change_infl_sm = ols_change_infl.predict(X_test_change_infl_sm)
mse_change_infl_sm = mean_squared_error(y_test_change_infl, y_pred_change_infl_sm)


In [20]:
# Ridge Regression for rate of inflation
ridge_reg_infl = Ridge(alpha=1.0)
ridge_reg_infl.fit(X_train_infl, y_train_infl)
y_pred_ridge_infl = ridge_reg_infl.predict(X_test_infl)
mse_ridge_infl = mean_squared_error(y_test_infl, y_pred_ridge_infl)

# Ridge Regression for change in inflation
ridge_reg_change_infl = Ridge(alpha=1.0)
ridge_reg_change_infl.fit(X_train_change_infl, y_train_change_infl)
y_pred_ridge_change_infl = ridge_reg_change_infl.predict(X_test_change_infl)
mse_ridge_change_infl = mean_squared_error(y_test_change_infl, y_pred_ridge_change_infl)


In [29]:


# Display the MSE for both models
print(f'\nLinear Regression (OLS) MSE (Inflation): {mse_infl_sm}')
print(f'Linear Regression (OLS) MSE (Change in Inflation): {mse_change_infl_sm}')
print(f'Ridge Regression MSE (Inflation): {mse_ridge_infl}')
print(f'Ridge Regression MSE (Change in Inflation): {mse_ridge_change_infl}')




Linear Regression (OLS) MSE (Inflation): 5.486038279474197
Linear Regression (OLS) MSE (Change in Inflation): 20.453879645804495
Ridge Regression MSE (Inflation): 6.9337433995942295
Ridge Regression MSE (Change in Inflation): 3.299737560712552


In [22]:
# Display the OLS summary for inflation
print(f'OLS Regression Summary for Inflation:')
print(ols_infl.summary())

# Display the OLS summary for change in inflation
print(f'\nOLS Regression Summary for Change in Inflation:')
print(ols_change_infl.summary())



OLS Regression Summary for Inflation:
                            OLS Regression Results                            
Dep. Variable:      Rate of inflation   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.939
Method:                 Least Squares   F-statistic:                     18.43
Date:                Wed, 22 May 2024   Prob (F-statistic):             0.0526
Time:                        14:10:29   Log-Likelihood:                 1.2684
No. Observations:                  18   AIC:                             29.46
Df Residuals:                       2   BIC:                             43.71
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         

/home/rahul/.local/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1971: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  k, _ = kurtosistest(a, axis)


In [25]:


# Display the coefficients for Ridge Regression
print('\nRidge Regression Coefficients (Inflation):')
print(pd.Series(ridge_reg_infl.coef_, index=predictors + control_variables))

print('\nRidge Regression Coefficients (Change in Inflation):')
print(pd.Series(ridge_reg_change_infl.coef_, index=predictors + control_variables))




Ridge Regression Coefficients (Inflation):
Domestic liabilities of the Centre        -1.181050
External liabilities of the Centre        -0.949901
Total liabilities of the Centre           -1.172062
Gross fiscal deficit                       0.062891
Revenue deficit                            0.365478
Tax Revenue (Net)                         -0.079092
Direct Tax (Net)                           1.840831
Indirect Tax                              -1.787632
Non-Tax Revenue                            1.020084
Revenue Receipts                           0.073255
Capital Receipts                           1.240223
Total Receipts                             0.613344
Volume of Central Govt Dated Securities   -0.295258
CRR                                        1.094443
Repo Rate                                 -1.322331
dtype: float64

Ridge Regression Coefficients (Change in Inflation):
Domestic liabilities of the Centre        -0.131691
External liabilities of the Centre        -0.684111
Tot

In [27]:


# Display the predictions
print('\nLinear Regression (OLS) Predictions (Inflation):')
print(pd.DataFrame({'Actual': y_test_infl, 'Predicted': y_pred_infl_sm}))

print('\nLinear Regression (OLS) Predictions (Change in Inflation):')
print(pd.DataFrame({'Actual': y_test_change_infl, 'Predicted': y_pred_change_infl_sm}))




Linear Regression (OLS) Predictions (Inflation):
     Actual  Predicted
15   6.3729   7.098697
9   10.0179   9.315666
0    6.6990   1.705648
8    6.6657   7.388646
17   4.2463   3.269540

Linear Regression (OLS) Predictions (Change in Inflation):
    Actual  Predicted
15    0.58  -0.610734
9     0.54  -2.604202
0     1.57  -4.882828
8    -3.35   3.656513
17    0.48   0.965104


In [28]:


print('\nRidge Regression Predictions (Inflation):')
print(pd.DataFrame({'Actual': y_test_infl, 'Predicted': y_pred_ridge_infl}))

print('\nRidge Regression Predictions (Change in Inflation):')
print(pd.DataFrame({'Actual': y_test_change_infl, 'Predicted': y_pred_ridge_change_infl}))



Ridge Regression Predictions (Inflation):
     Actual  Predicted
15   6.3729   6.331802
9   10.0179   5.193975
0    6.6990   5.132896
8    6.6657   4.322697
17   4.2463   6.104911

Ridge Regression Predictions (Change in Inflation):
    Actual  Predicted
15    0.58  -0.310542
9     0.54  -0.163404
0     1.57  -0.805579
8    -3.35  -0.256936
17    0.48   0.459255
